# Disaster Declarations
Author: Mark Bauer

In [16]:
import duckdb

# Read In Data

In [17]:
con = duckdb.connect()

con.execute("""
    CREATE TABLE disaster_declarations
    AS FROM read_parquet('https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.parquet')
"""
)

sql = """
    SELECT *
    FROM disaster_declarations
    LIMIT 10
"""

con.sql(sql)

┌──────────────────────┬────────────────┬─────────┬───┬──────────────────────┬──────────────────────┐
│          id          │ disasterNumber │  state  │ … │     lastRefresh      │         hash         │
│       varchar        │     int16      │ varchar │   │      timestamp       │       varchar        │
├──────────────────────┼────────────────┼─────────┼───┼──────────────────────┼──────────────────────┤
│ f15a7a79-f1c3-41bb…  │           5530 │ NV      │ … │ 2024-08-27 18:22:1…  │ 5d07e7c51bb300bfbe…  │
│ 09e3f81a-5e16-4b72…  │           5529 │ OR      │ … │ 2024-08-27 18:22:1…  │ ae87cf3c6ed795015b…  │
│ 59983f89-30bf-4888…  │           5528 │ OR      │ … │ 2024-08-27 18:22:1…  │ 432cf0995c47e3895c…  │
│ 8d13ecf0-bc2f-496b…  │           5527 │ OR      │ … │ 2024-08-27 18:22:1…  │ 2f21d90cb6bc64b0d4…  │
│ 17c24d4a-49a9-4cac…  │           5526 │ CO      │ … │ 2024-08-27 18:22:1…  │ e753ba692156f389db…  │
│ f1140a27-cb85-404c…  │           5525 │ CO      │ … │ 2024-08-27 18:22:1…  │ b1f

# Describe and Summarize Data

In [18]:
sql = "DESCRIBE SELECT * FROM disaster_declarations"
describe_relation = con.sql(sql)

(con
 .sql("SELECT column_name, column_type FROM describe_relation")
 .show(max_rows=50)
)

┌──────────────────────────┬─────────────┐
│       column_name        │ column_type │
│         varchar          │   varchar   │
├──────────────────────────┼─────────────┤
│ id                       │ VARCHAR     │
│ disasterNumber           │ SMALLINT    │
│ state                    │ VARCHAR     │
│ femaDeclarationString    │ VARCHAR     │
│ declarationType          │ VARCHAR     │
│ declarationDate          │ DATE        │
│ fyDeclared               │ SMALLINT    │
│ incidentType             │ VARCHAR     │
│ declarationTitle         │ VARCHAR     │
│ ihProgramDeclared        │ BOOLEAN     │
│ iaProgramDeclared        │ BOOLEAN     │
│ paProgramDeclared        │ BOOLEAN     │
│ hmProgramDeclared        │ BOOLEAN     │
│ incidentBeginDate        │ DATE        │
│ incidentEndDate          │ DATE        │
│ disasterCloseoutDate     │ DATE        │
│ tribalRequest            │ BOOLEAN     │
│ fipsStateCode            │ VARCHAR     │
│ fipsCountyCode           │ VARCHAR     │
│ placeCode

In [19]:
sql = "SUMMARIZE SELECT * FROM disaster_declarations"
summarize_relation = con.sql(sql)

summarize_relation.df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,00002fed-4c4b-49c0-8031-6e9749819fde,ffffdc28-ff85-43c7-91bb-f40fc4932471,65556,None,None,None,None,None,66941,0.00
1,disasterNumber,SMALLINT,1,5538,5081,2654.1278737993157,1428.1984623171707,1359,3133,3712,66941,0.00
2,state,VARCHAR,AK,WY,59,None,None,None,None,None,66941,0.00
3,femaDeclarationString,VARCHAR,DR-1-GA,FM-5538-CA,5017,None,None,None,None,None,66941,0.00
4,declarationType,VARCHAR,DR,FM,3,None,None,None,None,None,66941,0.00
5,declarationDate,DATE,1953-05-02,2024-10-01,3514,None,None,None,None,None,66941,0.00
6,fyDeclared,SMALLINT,1953,2025,74,2005.1649362871783,14.987269739159135,1997,2008,2019,66941,0.00
7,incidentType,VARCHAR,Biological,Winter Storm,26,None,None,None,None,None,66941,0.00
8,declarationTitle,VARCHAR,HURRICANE EARL,ZOGG FIRE,2349,None,None,None,None,None,66941,0.00
9,ihProgramDeclared,BOOLEAN,false,true,2,None,None,None,None,None,66941,0.00


In [20]:
sql = """
    SELECT column_name, column_type, null_percentage
    FROM summarize_relation
    ORDER BY null_percentage DESC
"""

con.sql(sql).show(max_rows=50)

┌──────────────────────────┬─────────────┬─────────────────┐
│       column_name        │ column_type │ null_percentage │
│         varchar          │   varchar   │  decimal(9,2)   │
├──────────────────────────┼─────────────┼─────────────────┤
│ lastIAFilingDate         │ DATE        │           71.93 │
│ designatedIncidentTypes  │ VARCHAR     │           71.42 │
│ disasterCloseoutDate     │ DATE        │           23.53 │
│ incidentEndDate          │ DATE        │            1.65 │
│ id                       │ VARCHAR     │            0.00 │
│ disasterNumber           │ SMALLINT    │            0.00 │
│ state                    │ VARCHAR     │            0.00 │
│ femaDeclarationString    │ VARCHAR     │            0.00 │
│ declarationType          │ VARCHAR     │            0.00 │
│ declarationDate          │ DATE        │            0.00 │
│ fyDeclared               │ SMALLINT    │            0.00 │
│ incidentType             │ VARCHAR     │            0.00 │
│ declarationTitle      

# Preview Values

In [21]:
sql = """
    SELECT
        id,
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        declarationDate,
        fyDeclared,
        incidentType,
        declarationTitle,
        ihProgramDeclared,
        iaProgramDeclared,
        paProgramDeclared,
        hmProgramDeclared
    FROM
        disaster_declarations
    LIMIT 5   
"""

con.sql(sql).df()

,id,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,f15a7a79-f1c3-41bb-8a5c-c05fbae34423,5530,NV,FM-5530-NV,FM,2024-08-12,2024,Fire,GOLD RANCH FIRE,False,False,True,True
1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c,5529,OR,FM-5529-OR,FM,2024-08-09,2024,Fire,LEE FALLS FIRE,False,False,True,True
2,59983f89-30bf-4888-b21b-62e8d57d9aac,5528,OR,FM-5528-OR,FM,2024-08-06,2024,Fire,ELK LANE FIRE,False,False,True,True
3,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0,5527,OR,FM-5527-OR,FM,2024-08-02,2024,Fire,MILE MARKER 132 FIRE,False,False,True,True
4,17c24d4a-49a9-4cac-9322-e5427c4cdfeb,5526,CO,FM-5526-CO,FM,2024-08-01,2024,Fire,QUARRY FIRE,False,False,True,True


In [22]:
sql = """
    SELECT
        incidentBeginDate,
        incidentEndDate,
        disasterCloseoutDate,
        tribalRequest,
        fipsStateCode,
        fipsCountyCode,
        placeCode,
        designatedArea,
        declarationRequestNumber,
        lastIAFilingDate,
        incidentId,
        region,
        designatedIncidentTypes,
        lastRefresh,
        hash
    FROM
        disaster_declarations
    LIMIT 5  
"""

con.sql(sql).df()

,incidentBeginDate,incidentEndDate,disasterCloseoutDate,tribalRequest,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash
0,2024-08-11,NaT,NaT,False,32,031,99031,Washoe (County),24123,NaT,2024081201,9,R,2024-08-27 18:22:14.800,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd
1,2024-08-08,NaT,NaT,False,41,067,99067,Washington (County),24122,NaT,2024081001,10,R,2024-08-27 18:22:14.800,ae87cf3c6ed795015b714af7166c7c295b2b67c7
2,2024-08-04,NaT,NaT,False,41,031,99031,Jefferson (County),24116,NaT,2024080701,10,R,2024-08-27 18:22:14.800,432cf0995c47e3895cea696ede5621b810460501
3,2024-08-02,NaT,NaT,False,41,017,99017,Deschutes (County),24111,NaT,2024080301,10,R,2024-08-27 18:22:14.800,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa
4,2024-07-30,NaT,NaT,False,08,059,99059,Jefferson (County),24106,NaT,2024080102,8,R,2024-08-27 18:22:14.800,e753ba692156f389dbe19f7a1c332d04ae145f74


# Analysis

In [25]:
sql = """
    SELECT
       fyDeclared AS year,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        fyDeclared
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────┬───────┐
│ year  │ count │
│ int16 │ int64 │
├───────┼───────┤
│  2020 │  9490 │
│  2005 │  4692 │
│  2011 │  2684 │
│  2008 │  2430 │
│  2021 │  2156 │
│  2024 │  1999 │
│  2019 │  1910 │
│  2004 │  1863 │
│  2017 │  1835 │
│  1993 │  1773 │
│  1996 │  1750 │
│  2003 │  1673 │
│  2010 │  1637 │
│  1998 │  1487 │
│  2009 │  1456 │
│  1999 │  1384 │
│  2001 │  1372 │
│  2023 │  1338 │
│  2007 │  1337 │
│  1977 │  1197 │
├───────┴───────┤
│    20 rows    │
└───────────────┘

In [26]:
sql = """
    SELECT
       state,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        state
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌─────────┬───────┐
│  state  │ count │
│ varchar │ int64 │
├─────────┼───────┤
│ TX      │  5350 │
│ KY      │  2762 │
│ MO      │  2750 │
│ FL      │  2649 │
│ GA      │  2600 │
│ LA      │  2586 │
│ VA      │  2572 │
│ OK      │  2543 │
│ NC      │  2297 │
│ PR      │  2078 │
│ MS      │  1945 │
│ IA      │  1922 │
│ KS      │  1839 │
│ AL      │  1742 │
│ TN      │  1709 │
│ CA      │  1678 │
│ AR      │  1624 │
│ MN      │  1621 │
│ NE      │  1533 │
│ NY      │  1532 │
├─────────┴───────┤
│     20 rows     │
└─────────────────┘

In [27]:
sql = """
    SELECT
       state,
       designatedArea,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        state, designatedArea
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌─────────┬─────────────────────────┬───────┐
│  state  │     designatedArea      │ count │
│ varchar │         varchar         │ int64 │
├─────────┼─────────────────────────┼───────┤
│ TX      │ Statewide               │   103 │
│ CA      │ Los Angeles (County)    │    82 │
│ CA      │ Riverside (County)      │    65 │
│ CA      │ San Bernardino (County) │    61 │
│ LA      │ St. Martin (Parish)     │    53 │
│ LA      │ Lafourche (Parish)      │    52 │
│ LA      │ Livingston (Parish)     │    51 │
│ FL      │ Collier (County)        │    51 │
│ CA      │ San Diego (County)      │    51 │
│ CA      │ Ventura (County)        │    51 │
│ LA      │ Terrebonne (Parish)     │    51 │
│ LA      │ St. Charles (Parish)    │    50 │
│ LA      │ Jefferson (Parish)      │    49 │
│ WA      │ Chelan (County)         │    49 │
│ LA      │ Assumption (Parish)     │    48 │
│ LA      │ Ascension (Parish)      │    48 │
│ FL      │ Bay (County)            │    48 │
│ LA      │ St. Tammany (Parish)  

In [28]:
sql = """
    SELECT
       incidentType,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        incidentType
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────────────────┬───────┐
│   incidentType    │ count │
│      varchar      │ int64 │
├───────────────────┼───────┤
│ Severe Storm      │ 18368 │
│ Hurricane         │ 13502 │
│ Flood             │ 11090 │
│ Biological        │  7857 │
│ Fire              │  3733 │
│ Snowstorm         │  3707 │
│ Severe Ice Storm  │  2942 │
│ Tornado           │  1623 │
│ Drought           │  1292 │
│ Tropical Storm    │   912 │
│ Coastal Storm     │   637 │
│ Other             │   313 │
│ Freezing          │   301 │
│ Earthquake        │   228 │
│ Typhoon           │   130 │
│ Winter Storm      │   117 │
│ Volcanic Eruption │    51 │
│ Mud/Landslide     │    42 │
│ Fishing Losses    │    42 │
│ Dam/Levee Break   │    13 │
├───────────────────┴───────┤
│ 20 rows         2 columns │
└───────────────────────────┘

In [29]:
sql = """
    SELECT
       fyDeclared AS year,
       incidentType,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        year, incidentType
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────┬──────────────────┬───────┐
│ year  │   incidentType   │ count │
│ int16 │     varchar      │ int64 │
├───────┼──────────────────┼───────┤
│  2020 │ Biological       │  7855 │
│  2005 │ Hurricane        │  3715 │
│  2011 │ Severe Storm     │  1232 │
│  2008 │ Severe Storm     │  1146 │
│  2017 │ Hurricane        │  1144 │
│  2004 │ Severe Storm     │  1071 │
│  2010 │ Severe Storm     │  1053 │
│  2021 │ Severe Ice Storm │   952 │
│  2007 │ Severe Storm     │   930 │
│  2009 │ Severe Storm     │   919 │
│  1998 │ Severe Storm     │   916 │
│  2003 │ Severe Storm     │   905 │
│  2020 │ Hurricane        │   858 │
│  2024 │ Tropical Storm   │   776 │
│  2021 │ Hurricane        │   771 │
│  1993 │ Snowstorm        │   771 │
│  1977 │ Drought          │   688 │
│  2019 │ Flood            │   651 │
│  2019 │ Hurricane        │   616 │
│  2004 │ Hurricane        │   616 │
├───────┴──────────────────┴───────┤
│ 20 rows                3 columns │
└──────────────────────────────────┘

In [30]:
sql = """
    SELECT
       state,
       incidentType,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        state, incidentType
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌─────────┬──────────────┬───────┐
│  state  │ incidentType │ count │
│ varchar │   varchar    │ int64 │
├─────────┼──────────────┼───────┤
│ LA      │ Hurricane    │  1447 │
│ MO      │ Severe Storm │  1385 │
│ TX      │ Hurricane    │  1370 │
│ NC      │ Hurricane    │  1304 │
│ FL      │ Hurricane    │  1290 │
│ TX      │ Fire         │  1229 │
│ KY      │ Severe Storm │  1156 │
│ KS      │ Severe Storm │  1092 │
│ OK      │ Severe Storm │  1041 │
│ PR      │ Hurricane    │  1024 │
│ GA      │ Hurricane    │   978 │
│ VA      │ Hurricane    │   877 │
│ TN      │ Severe Storm │   845 │
│ TX      │ Severe Storm │   760 │
│ MS      │ Hurricane    │   745 │
│ IA      │ Severe Storm │   744 │
│ NE      │ Severe Storm │   730 │
│ IA      │ Flood        │   729 │
│ MN      │ Flood        │   719 │
│ MS      │ Severe Storm │   681 │
├─────────┴──────────────┴───────┤
│ 20 rows              3 columns │
└────────────────────────────────┘

In [31]:
sql = """
    SELECT
       state,
       designatedArea,
       incidentType,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        state, designatedArea, incidentType
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌─────────┬───────────────────────────────┬──────────────┬───────┐
│  state  │        designatedArea         │ incidentType │ count │
│ varchar │            varchar            │   varchar    │ int64 │
├─────────┼───────────────────────────────┼──────────────┼───────┤
│ TX      │ Statewide                     │ Fire         │    92 │
│ CA      │ Los Angeles (County)          │ Fire         │    55 │
│ CA      │ Riverside (County)            │ Fire         │    37 │
│ NV      │ Washoe (County)               │ Fire         │    34 │
│ CA      │ San Bernardino (County)       │ Fire         │    33 │
│ WA      │ Chelan (County)               │ Fire         │    32 │
│ LA      │ Plaquemines (Parish)          │ Hurricane    │    32 │
│ LA      │ Lafourche (Parish)            │ Hurricane    │    31 │
│ LA      │ Terrebonne (Parish)           │ Hurricane    │    31 │
│ LA      │ St. Charles (Parish)          │ Hurricane    │    31 │
│ LA      │ Iberia (Parish)               │ Hurricane    │    

In [32]:
sql = """
    SELECT
       declarationType,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY
        declarationType
    ORDER BY
        count DESC   
"""

con.sql(sql)

┌─────────────────┬───────┐
│ declarationType │ count │
│     varchar     │ int64 │
├─────────────────┼───────┤
│ DR              │ 45001 │
│ EM              │ 19961 │
│ FM              │  1979 │
└─────────────────┴───────┘